In [56]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

from datetime import date
print(date.today())

2025-12-07


folder contents
```bash
dataset_nc2017kremer
├── [9.4M]  S1_41467_2017_BFncomms15824_MOESM390_ESM.txt
├── [ 30K]  S2_41467_2017_BFncomms15824_MOESM391_ESM.txt  
├── [5.2K]  S3_41467_2017_BFncomms15824_MOESM392_ESM.txt   
├── [204K]  S6_41467_2017_BFncomms15824_MOESM395_ESM.txt  
├── [ 95K]  S7_41467_2017_BFncomms15824_MOESM396_ESM.txt  
├── [4.1K]  S8_41467_2017_BFncomms15824_MOESM397_ESM.txt  
```

In [3]:
workdir = '/mnt/disk7t/xwj/axolotl_rev'
datasetdir = f'{workdir}/dataset_nc2017kremer'

os.chdir(datasetdir)

### preprocessing

In [33]:
file = 'S1_41467_2017_BFncomms15824_MOESM390_ESM.txt' # RNA read count table. (TXT 9604 kb)
cts = pd.read_csv(file, sep='\t', index_col=0)
cts.index.name, cts.columns.name = 'Gene', 'Sample'
# cts columns are RNA_ID, index are gene names.
print(cts.shape)

# whole sample set
min_reads = 5
cts14k = cts[ (cts > min_reads).any(axis=1) ].copy() # 14k genes
print(cts14k.shape)

# subset use non-zero genes only
cts11k = cts.loc[ (cts > 0).all(axis=1) ].copy() # 11k genes
cts11k.index = [ g.replace(' ','') for g in cts11k.index ]
cts11k.index.name, cts11k.columns.name = 'Gene', 'Sample'
cts11k.columns = [f'X{s}' if not s.startswith('MUC') else s for s in cts11k.columns]

print(cts11k.shape)

(27682, 119)
(14409, 119)
(11048, 119)


## NMD genes + 6 valid genes

In [11]:
# 这里有19个DNA提示NMD的Fibroblast样本,共20个基因（其中FB71215样本有两个基因)
# RNAseq提示Aberrant Expression的样本不在其中
file = 'S3_41467_2017_BFncomms15824_MOESM392_ESM.txt' # Sample annotation. (TXT 5 kb). 
# 'DIAGNOSIS_GENE'
# 'EXOME_ENRICHMENT'
# 'HAS_EXOME'
# 'HAS_PROTEOME',
# 'VARIANT_EFFECT' ( )
# 'RNA_DEFECT_FOR_DIAGNOSIS_GENE' # 判定为诊断基因的那些基因名称 aberrant_splicing 8,  MAE  4, aberrant_expression  4
# 'NUM_WES_SIGNI',
# 'NUM_RNA_ABER_EXP_SIGNI'
# 'NUM_RNA_ABER_SPLICE_SIGNI'
# 'NUM_MAE_SIGNI'

tab2 = pd.read_csv(file, sep='\t', index_col=0)

key_effect = ['stop','frame-shift']
tab2['pred_effect'] = None
for ke in key_effect:
    for idx, row in tab2.iterrows():
        # 确保 VARIANT_EFFECT 是字符串类型
        variant_effect = str(row['VARIANT_EFFECT'])
        if ke in variant_effect:
            tab2.loc[idx, 'pred_effect'] = 'nmd'


# Part 1 of potential expression outliers
cols = ['DIAGNOSIS_GENE', 'pred_effect','VARIANT_EFFECT',  'RNA_DEFECT_FOR_DIAGNOSIS_GENE']
selected_cols = cols[1:]
outlier_sample_col = 'Sample'
outlier_gene_col = 'Gene'
pred_nmd = tab2.query('pred_effect== "nmd"')[cols].copy()
# for cols in ['VARIANT_EFFECT', 'pred_effect', 'RNA_DEFECT_FOR_DIAGNOSIS_GENE']:

result = [
    ';'.join([f"{col}={row[col] if pd.notna(row[col]) else None }" for col in selected_cols])
    for _, row in pred_nmd.iterrows()
]
pred_nmd = pred_nmd.reset_index().rename(columns = {'DIAGNOSIS_GENE': outlier_gene_col, 'FIBROBLAST_ID': outlier_sample_col}).drop(columns=selected_cols)
pred_nmd['comment'] = result
pred_nmd[outlier_sample_col] = 'FB' + pred_nmd[outlier_sample_col].astype(str)

# 一个病人有两个基因被判定为诊断基因，需要复制为two rows
new_row = []
del_row_idx = []
for idx, row in pred_nmd.iterrows():
    if ";" in row['Gene']:
        g1, g2 = row['Gene'].split(";")
        # Sample	Gene	comment
        new_row.append([row['Sample'], g1, row['comment']])
        new_row.append([row['Sample'], g2, row['comment']])
        del_row_idx.append(idx)
        print(idx,g1, g2)
        
pred_nmd = pd.concat([pred_nmd.drop(del_row_idx), 
                      pd.DataFrame(new_row, columns = pred_nmd.columns)], 
                     ignore_index=True)
pred_nmd = pred_nmd[pred_nmd['Gene'].isin(cts.index)].reset_index(drop=True)
pred_nmd

8 LYRM7 MTO1


,Sample,Gene,comment
0,FB49665,GTPBP3,pred_effect=nmd;VARIANT_EFFECT=missense;frame-...
1,FB58027,NDUFB3,pred_effect=nmd;VARIANT_EFFECT=missense;stop;R...
2,FB60592,MGME1,pred_effect=nmd;VARIANT_EFFECT=stop;RNA_DEFECT...
3,FB61691,DNAJC3,pred_effect=nmd;VARIANT_EFFECT=stop;RNA_DEFECT...
4,FB61695,DNAJC3,pred_effect=nmd;VARIANT_EFFECT=stop;RNA_DEFECT...
5,FB61818,SFXN4,pred_effect=nmd;VARIANT_EFFECT=frame-shift;spl...
6,FB68607,CDKL5,pred_effect=nmd;VARIANT_EFFECT=frame-shift;RNA...
7,FB73897,NBAS,pred_effect=nmd;VARIANT_EFFECT=frame-shift;fra...
8,FB73898,IARS,pred_effect=nmd;VARIANT_EFFECT=missense;stop;R...
9,FB75110,NBAS,pred_effect=nmd;VARIANT_EFFECT=missense;stop;R...


In [34]:
# undiagnosed patients. Part 2 of potential expression outliers
file = 'S7_41467_2017_BFncomms15824_MOESM396_ESM.txt' # Candidate genes for undiagnosed patients for all 3 strategies. (TXT 94 kb)
# tab7 have fibroblast ids only 
tab7 = pd.read_csv(file, sep='\t', index_col=0)

file = 'S8_41467_2017_BFncomms15824_MOESM397_ESM.txt' # RNA sample annotation for read count normalization. (TXT 4 kb)
# used for sample id mapping. fibroblast id to RNA_ID
tab8 = pd.read_csv(file, sep='\t', index_col=0)
tab8['FB_ID'] = 'FB' + tab8.index.astype(str)
tab8['RNA_ID'] = [f'X{s}' if not s.startswith('MUC') else s for s in tab8['RNA_ID']]
tab8 = tab8.set_index('FB_ID')#.loc[71215,:]
print(tab8.shape)
# part 2 of potential expression outliers. DNA and Protein validation evidence.

genes = ["TIMMDC1", "MGST1", "CLPP", "ALDH18A1","MCOLN1"] 
# The above cases validated by Proteomics/western blot as described in Nature Communications (2017). also discussed  by OUTRIDER
cols = ['HGNCID', "WES_SIGNI",'RNA_ABER_EXP_SIGNI', 'MAE_IS_SIGNI', 'RNA_ABER_SPLICE_SIGNI']
selected_cols = cols[1:] 
outlier_sample_col = 'Sample'
outlier_gene_col = 'Gene'

val_protein = tab7.query('HGNCID in @genes')[cols].copy()

result = [
    ';'.join([f"{col}={row[col] if pd.notna(row[col]) else None }" for col in selected_cols])
    for _, row in val_protein.iterrows()
]
val_protein = val_protein.reset_index().rename(columns = {'HGNCID': outlier_gene_col, 'FIBROBLAST_ID': outlier_sample_col}).drop(columns=selected_cols)
val_protein['comment'] = result
val_protein[outlier_sample_col] = 'FB' + val_protein[outlier_sample_col].astype(str)
val_protein.loc[len(val_protein)] = [ tab8.query('RNA_ID == "MUC1361"').index.values[0], 'MCOLN1','AGE slightly inferior of the thresholds']
val_protein

(119, 4)


,Sample,Gene,comment
0,FB58955,CLPP,WES_SIGNI=True;RNA_ABER_EXP_SIGNI=False;MAE_IS...
1,FB35791,TIMMDC1,WES_SIGNI=None;RNA_ABER_EXP_SIGNI=True;MAE_IS_...
2,FB66744,TIMMDC1,WES_SIGNI=None;RNA_ABER_EXP_SIGNI=True;MAE_IS_...
3,FB73804,MGST1,WES_SIGNI=None;RNA_ABER_EXP_SIGNI=True;MAE_IS_...
4,FB80256,ALDH18A1,WES_SIGNI=True;RNA_ABER_EXP_SIGNI=True;MAE_IS_...
5,FB62346,MCOLN1,AGE slightly inferior of the thresholds


# Write counts file & outlier file & task config file

In [40]:
# use 11k
cts = cts11k.copy()

In [41]:
def func(raw): # 改成用RNA_ID n=119
    df_outlier = tab8.loc[raw['Sample'],:].copy()
    for idx, row in df_outlier.iterrows():
        df_outlier.loc[idx, ['Gene','comment']]  = raw.set_index('Sample').loc[row.name].values
    df_outlier = df_outlier.reset_index().rename(columns={'RNA_ID':'Sample'})
    return df_outlier

#  Part 1 of potential expression outliers
# outlier_pred = pred_nmd
outlier_pred = func(pred_nmd)
outlier_val = func(val_protein)
#  Part 2 of potential expression outliers. DNA and Protein validation evidence.
outlier_pred_and_val = func(pd.concat([pred_nmd, val_protein], ignore_index=True))

# df_outlier = tab8.loc[outlier_pred_and_val['Sample'],:].copy()
# for idx, row in df_outlier.iterrows():
#     df_outlier.loc[idx, ['Gene','comment']]  = outlier_pred_and_val.set_index('Sample').loc[row.name].values
# df_outlier = df_outlier.reset_index().rename(columns={'RNA_ID':'Sample'})

assert outlier_pred_and_val['Sample'].isin(cts.columns).all()
assert outlier_pred['Sample'].isin(cts.columns).all()
print(outlier_pred_and_val.shape, outlier_pred.shape, outlier_val.shape)
# outlier_pred_and_val = df_outlier.copy()

# ctsfile = f'{workdir}/gtex_processed/cts_{tissue_id}_s{cts.shape[1]}_g{cts.shape[0]}.tsv.gz'
# outlierfile = f'{workdir}/gtex_processed/outlier_{tissue_id}_sg{outlier.shape[0]}.tsv'
tissue_id = 'FB'

(28, 7) (22, 7) (6, 7)


In [459]:
outlierfile3 =f'{datasetdir}/outlier_{tissue_id}_sg{outlier_val.shape[0]}.tsv'
print(outlierfile3)
outlier_val.to_csv(outlierfile3, sep='\t')

/mnt/disk7t/xwj/axolotl_rev/dataset_nc2017kremer/outlier_FB_sg6.tsv


In [496]:
# Table output
outlier_pred_and_val.to_excel(f'{workdir}/result/table/s2_kremer119_outlier28.xlsx', 
                              sheet_name='s2_kremer119_outlier28',index=True)
outlier_pred_and_val.head()

,FB_ID,Sample,SEX,RNA_HOX_GROUP,RNA_BATCH_GROUP,Gene,comment
0,FB49665,X74172,male,magenta,orange,GTPBP3,pred_effect=nmd;VARIANT_EFFECT=missense;frame-...
1,FB58027,X76630,female,magenta,purple,NDUFB3,pred_effect=nmd;VARIANT_EFFECT=missense;stop;R...
2,FB60592,X76622,male,magenta,purple,MGME1,pred_effect=nmd;VARIANT_EFFECT=stop;RNA_DEFECT...
3,FB61691,X76636,male,magenta,darkgreen,DNAJC3,pred_effect=nmd;VARIANT_EFFECT=stop;RNA_DEFECT...
4,FB61695,X61695,male,magenta,orange,DNAJC3,pred_effect=nmd;VARIANT_EFFECT=stop;RNA_DEFECT...


In [43]:
cts.loc[outlier_val['Gene'], outlier_val['Sample'] ]

,MUC1350,MUC1344,MUC1365,MUC1396,MUC1404,MUC1361
Gene,,,,,,
CLPP,581,980,1197,1337,991,940
TIMMDC1,2500,300,282,2403,2568,2303
TIMMDC1,2500,300,282,2403,2568,2303
MGST1,4432,3955,3960,373,7887,6760
ALDH18A1,5664,6268,3704,5878,2642,4810
MCOLN1,700,356,662,1061,1138,150


In [50]:
for use in ['t00_FB_s119_g14409', 't00_FB_s119_g11048']:
    if use == 't00_FB_s119_g14409':
        cts = cts14k.copy()
    elif use == 't00_FB_s119_g11048': # use = 't00_FB_s119_g11048' # all genes have minreads 1, subset random set from this cts table
        cts = cts11k.copy()
    print(use, cts.shape)
    tissue_name_mapping = pd.DataFrame({
        'TISSUE_ID': ['FB'],
        'TISSUE_NAME': ['Fibroblast'],
    })

    tissues = pd.DataFrame(
        index = tissue_name_mapping.index,
        columns = ['TISSUE_ID','TISSUE_NAME','N_SAMPLE','N_GENE','CTS_FILE', 'OUTLIER_FILE',])

    # 各组织cts表达矩阵和outlier
    for idx, row in tissue_name_mapping.iterrows():
        tissue_id, tissue_name = row['TISSUE_ID'],  row['TISSUE_NAME']
        
        # (1) cts: expression matrix
        # (2) outlier: true outlier gene-sample pair in cts.
        ctsfile = f'{datasetdir}/cts_{tissue_id}_s{cts.shape[1]}_g{cts.shape[0]}.tsv.gz'
        cts.to_csv(ctsfile, sep='\t')
        # outliers: part1 + part2
        outlierfile =f'{datasetdir}/outlier_{tissue_id}_sg{outlier_pred_and_val.shape[0]}.tsv'
        outlier_pred_and_val.to_csv(outlierfile, sep='\t')
        # outliers: part1 only
        outlierfile2 =f'{datasetdir}/outlier_{tissue_id}_sg{outlier_pred.shape[0]}.tsv'
        outlier_pred.to_csv(outlierfile2, sep='\t')
        
        print(f'{tissue_name}/{tissue_id}/min_reads={min_reads} stable_ngene={cts.shape[0]} nsample={cts.shape[1]}')

        # print(outlierfile,ctsfile)
        tissues.loc[idx, :] = [ tissue_id, tissue_name, cts.shape[1], cts.shape[0],ctsfile, outlierfile]
        
    tissues.to_csv(f'tissues_{use}.tsv', sep='\t')
    tissues.values

t00_FB_s119_g14409 (14409, 119)
Fibroblast/FB/min_reads=5 stable_ngene=14409 nsample=119
t00_FB_s119_g11048 (11048, 119)
Fibroblast/FB/min_reads=5 stable_ngene=11048 nsample=119


In [53]:
use = 't00_FB_s119_g14409' # all genes have minreads 1, subset random set from here

tissues = pd.read_csv(f'tissues_{use}.tsv', sep='\t', index_col=0)
tissues 

,TISSUE_ID,TISSUE_NAME,N_SAMPLE,N_GENE,CTS_FILE,OUTLIER_FILE
0,FB,Fibroblast,119,14409,/mnt/disk7t/xwj/axolotl_rev/dataset_nc2017krem...,/mnt/disk7t/xwj/axolotl_rev/dataset_nc2017krem...


In [52]:
# level 1
output_path = f'{workdir}/result/dataset_nc2017kremer'
# level 2
samples_path = f'{output_path}/samples'
task_config_path = f'{output_path}/task_config'
task_output_path = f'{output_path}/task_output'
metric_output_path = f'{output_path}/metric'

# print(workdir, samples_path, task_config_path, task_output_path, metric_output_path)
os.system(f'mkdir -p {samples_path} {task_config_path} {task_output_path} {metric_output_path}')
# os.system(f'chmod --silent -R 777 {task_output_path}')

0

In [ ]:
# prepare task config file.
# task_config table have columns indicating output filenames of different methods

for i, row in tissues.iterrows():
    
    t  = row.TISSUE_ID
    tn = row.TISSUE_NAME
    ns = row.N_SAMPLE
    ng = row.N_GENE
    ctsfile = row.CTS_FILE
    outlierfile = row.OUTLIER_FILE

    prefix = f't{i:02d}_{t}_s{ns}_g{ng}' # id, number of samples, number of genes.
    
    cols = ['Dname','cts','samples','MyMethod','OUTRIDER','ABEILLE','OUTSINGLE']
    tasks = [0] # list of parallel tasks
    task_config = pd.DataFrame(index=tasks, columns=cols)
    task_config.index.name = 'task'

    task = 0 # 默认是全部样本, 所以only one task for this tissue  
    task_config.loc[task, 'Dname' ] = t
    task_config.loc[task, 'cts' ] = ctsfile
    task_config.loc[task, 'samples' ] = f'{samples_path}/{prefix}.txt'
    # create filenames
    task_config.loc[task, 'MyMethod'] = f'{task_output_path}/{prefix}/{task:03d}_mymethod.txt.gz'
    task_config.loc[task, 'OUTRIDER'] = f'{task_output_path}/{prefix}/{task:03d}_outrider.txt.gz'
    task_config.loc[task, 'OUTSINGLE'] = f'{task_output_path}/{prefix}/{task:03d}_outsingle.txt.gz'
    task_config.loc[task, 'ABEILLE'] = f'{task_output_path}/{prefix}/{task:03d}_abeille.txt.gz'

    # 0. create config & output folder of parallel tasks
    task_config.to_csv(f'{task_config_path}/{prefix}.config',sep='\t')
    os.system(f'mkdir -p {task_output_path}/{prefix}')
    
    # 1. sample ids of parallel tasks
    # 将task specific样本列表作为one row添加到DataFrame中. 默认task是全部样本
    cts = pd.read_csv(ctsfile, sep='\t',index_col=0)
    all_samples_df = pd.DataFrame(data=cts.columns.T.tolist()).transpose()
    all_samples_df.index = task_config.index
    all_samples_df.to_csv(f'{samples_path}/{prefix}.txt',sep='\t')
    

## subsampling to small sizes (min_reads = 0)

In [19]:
# positive samples
part1_6 = outlier_val.copy()
part2_18 = outlier_pred.drop_duplicates('Sample').copy()
part2_18.shape, part1_6.shape

((21, 7), (6, 7))

In [60]:
use = 't00_FB_s119_g11048'
tissues = pd.read_csv(f'tissues_{use}.tsv', sep='\t', index_col=0)
f'tissues_{use}.tsv'

'tissues_t00_FB_s119_g11048.tsv'

In [61]:
pwd

'/mnt/disk7t/xwj/axolotl_rev/dataset_nc2017kremer'

In [62]:
# 定义样本量列表和随机种子
sample_sizes = [100, 60, 30, 10]
n_replicates = 10  # 每个样本量重复10次
random_seeds = range(n_replicates)  # 使用0-9作为随机种子

# 从tissues中获取信息
tissue_data = tissues.iloc[0]
t = tissue_data[0]  # TISSUE_ID
tn = tissue_data[1]  # TISSUE_NAME
ns = tissue_data[2]  # N_SAMPLE
ng = tissue_data[3]  # N_GENE
ctsfile = tissue_data[4]  # CTS_FILE
outlierfile = tissue_data[5]  # OUTLIER_FILE

# 创建基础前缀
# base_prefix = f't00_{t}_s{ns}_g{ng}' # Version1.  six valid must in sub samples 
base_prefix = f't01_{t}_s{ns}_g{ng}_nmd' # Version2.  six valid + equal number of NMD, must in sub samples

cts = pd.read_csv(ctsfile, sep='\t', index_col=0)
all_samples = cts.columns.tolist()

# positive samples
part1_sampled = outlier_val.copy()
part2_18 = outlier_pred.drop_duplicates('Sample').copy()

# all negative samples
negative_samples = list(set(all_samples) - set(part1_sampled['Sample'].tolist()) - set(part2_18['Sample'].tolist()) )

# 创建字典存储所有抽样结果
sampled_configs = {}

# 对每个样本量进行抽样
for size in sample_sizes:
    # 存储当前样本量的所有抽样结果
    size_configs = {}
    
    for seed in random_seeds:
        if size <= 10:
            part2_sampled = part2_18.sample(n = size - len(part1_sampled), random_state=seed)
        else:
            part2_sampled = part2_18.copy()
            
        n_negative = size - len(part1_sampled) - len(part2_sampled)
        if n_negative: # 从阴性样本中随机抽样（数量 = 总样本量 - 阳性样本数量
            # negative_selected =  np.random.choice(negative_samples, size=n_negative, replace=False)
            negative_selected = pd.Series(negative_samples).sample(n = n_negative, random_state=seed).tolist() #, size=n_negative, replace=False)
        else:
            negative_selected = []
        # 合并阳性样本和随机选择的阴性样本
        selected_samples =  part1_sampled['Sample'].tolist() + part2_sampled['Sample'].tolist() + negative_selected
        assert len(selected_samples), len(set(selected_samples))
        print(size, seed, part1_sampled.shape, part2_sampled.shape, len(negative_selected), len(selected_samples))
        
        # 创建前缀
        prefix = f'{base_prefix}_size{size}_seed{seed}'
        
        # 创建输出目录
        output_dir = os.path.join(task_output_path, prefix)
        os.makedirs(output_dir, exist_ok=True)
        
        # 创建样本列表文件
        samples_file = os.path.join(samples_path, f'{prefix}.txt')
        samples = pd.DataFrame(selected_samples).transpose()
        samples.index.name = 'task'
        samples.to_csv(samples_file, sep='\t', header=True)
        
        # 创建config文件
        cols = ['Dname', 'cts', 'samples', 'MyMethod', 'OUTRIDER', 'ABEILLE', 'OUTSINGLE']
        task_config = pd.DataFrame(columns=cols)
        task_config.index.name = 'task'
        # 填充config数据
        task = 0
        task_config.loc[0, 'Dname'] = t
        task_config.loc[0, 'cts'] = ctsfile
        task_config.loc[0, 'samples'] = samples_file
        task_config.loc[0, 'MyMethod'] = os.path.join(output_dir, f'{task:03d}_mymethod.txt.gz')
        task_config.loc[0, 'OUTRIDER'] = os.path.join(output_dir, f'{task:03d}_outrider.txt.gz')
        task_config.loc[0, 'OUTSINGLE'] = os.path.join(output_dir, f'{task:03d}_outsingle.txt.gz')
        task_config.loc[0, 'ABEILLE'] = os.path.join(output_dir, f'{task:03d}_abeille.txt.gz')
        # {prefix}/{task:03d}
        
        # 保存config文件
        config_file = os.path.join(task_config_path, f'{prefix}.config')
        task_config.to_csv(config_file, sep='\t', index=False)
        
        # 存储结果
        size_configs[f'seed_{seed}'] = {
            'config': task_config,
            'selected_samples': selected_samples,
            'output_dir': output_dir
        }
    
    # 将当前样本量的所有结果存入总字典
    sampled_configs[f'size_{size}'] = size_configs

# sampled_configs现在包含了所有抽样结果和对应的配置文件
# 结构为：{sample_size: {seed: {'config': df, 'selected_samples': list, 'output_dir': str}}, ...}
output_file = os.path.join(task_config_path, f'{base_prefix}_sampled_configs_dict.pkl')
# 保存sampled_configs字典
with open(output_file, 'wb') as f:
    pickle.dump(sampled_configs, f)

    print(f"sampled_configs已保存到: {output_file}")


100 0 (6, 7) (21, 7) 73 100
100 1 (6, 7) (21, 7) 73 100
100 2 (6, 7) (21, 7) 73 100
100 3 (6, 7) (21, 7) 73 100
100 4 (6, 7) (21, 7) 73 100
100 5 (6, 7) (21, 7) 73 100
100 6 (6, 7) (21, 7) 73 100
100 7 (6, 7) (21, 7) 73 100
100 8 (6, 7) (21, 7) 73 100
100 9 (6, 7) (21, 7) 73 100
60 0 (6, 7) (21, 7) 33 60
60 1 (6, 7) (21, 7) 33 60
60 2 (6, 7) (21, 7) 33 60
60 3 (6, 7) (21, 7) 33 60
60 4 (6, 7) (21, 7) 33 60
60 5 (6, 7) (21, 7) 33 60
60 6 (6, 7) (21, 7) 33 60
60 7 (6, 7) (21, 7) 33 60
60 8 (6, 7) (21, 7) 33 60
60 9 (6, 7) (21, 7) 33 60
30 0 (6, 7) (21, 7) 3 30
30 1 (6, 7) (21, 7) 3 30
30 2 (6, 7) (21, 7) 3 30
30 3 (6, 7) (21, 7) 3 30
30 4 (6, 7) (21, 7) 3 30
30 5 (6, 7) (21, 7) 3 30
30 6 (6, 7) (21, 7) 3 30
30 7 (6, 7) (21, 7) 3 30
30 8 (6, 7) (21, 7) 3 30
30 9 (6, 7) (21, 7) 3 30
10 0 (6, 7) (4, 7) 0 10
10 1 (6, 7) (4, 7) 0 10
10 2 (6, 7) (4, 7) 0 10
10 3 (6, 7) (4, 7) 0 10
10 4 (6, 7) (4, 7) 0 10
10 5 (6, 7) (4, 7) 0 10
10 6 (6, 7) (4, 7) 0 10
10 7 (6, 7) (4, 7) 0 10
10 8 (6, 7) (4, 7)

### pct change from 0.08~0.24

In [63]:
import numpy as np
import pandas as pd
import os

# 定义样本量列表和随机种子
list_outlier_pct = [ 0.08, 0.16, 0.24,]
sample_sizes = [100, 50,]
n_replicates = 10  # 每个样本量重复10次
random_seeds = range(n_replicates)  # 使用0-9作为随机种子

# 从tissues中获取信息
tissue_data = tissues.iloc[0]
t = tissue_data[0]  # TISSUE_ID
tn = tissue_data[1]  # TISSUE_NAME
ns = tissue_data[2]  # N_SAMPLE
ng = tissue_data[3]  # N_GENE
ctsfile = tissue_data[4]  # CTS_FILE
outlierfile = tissue_data[5]  # OUTLIER_FILE

# 创建基础前缀
# base_prefix = f't00_{t}_s{ns}_g{ng}' # Version1.  six valid must in sub samples 
base_prefix = f't01_{t}_s{ns}_g{ng}_nmd' # Version2.  six valid + equal number of NMD, must in sub samples

cts = pd.read_csv(ctsfile, sep='\t', index_col=0)
all_samples = cts.columns.tolist()

# positive samples
part1_sampled = outlier_val.copy()
part2_18 = outlier_pred.drop_duplicates('Sample').copy()

# all negative samples
negative_samples = list(set(all_samples) - set(part1_sampled['Sample'].tolist()) - set(part2_18['Sample'].tolist()) )
i = 0
for pct in list_outlier_pct: # 增加pct的变化
    # 创建基础前缀
    base_prefix = f't{i:02d}_{t}_s{ns}_g{ng}_pct{pct:.2f}'
    print(base_prefix)
    # 创建字典存储所有抽样结果
    sampled_configs = {}

    # 对每个样本量进行抽样
    for size in sample_sizes:
        # 存储当前样本量的所有抽样结果
        size_configs = {}
        n_positive = int(size * pct) 
        n_negative = size - n_positive
        print(n_positive, n_negative)
        
        for seed in random_seeds:
            negative_selected = pd.Series(negative_samples).sample(n = n_negative, random_state=seed*1000+n_negative).tolist() 
            # include all part1_sampled. part1+part = n_positive
            if n_positive <= len(part1_sampled):
                part1_sampled_enough = part1_sampled.sample(n = n_positive, random_state=seed*1000+n_positive)
                part2_sampled = part2_18.sample(0)
                selected_samples =  part1_sampled_enough['Sample'].tolist() + negative_selected
            else:
                part2_sampled = part2_18.sample(n = n_positive - len(part1_sampled), random_state=seed*1000+n_positive)            
                # 合并阳性样本和随机选择的阴性样本
                selected_samples =  part1_sampled['Sample'].tolist() + part2_sampled['Sample'].tolist() + negative_selected
            assert len(selected_samples), len(set(selected_samples))
            # print(size, seed, len(part2_sampled), len(negative_selected), len(selected_samples))
            
            # 创建前缀
            prefix = f'{base_prefix}_size{size}_seed{seed}'
            
            # 创建输出目录
            output_dir = os.path.join(task_output_path, prefix)
            os.makedirs(output_dir, exist_ok=True)
            
            # 创建样本列表文件
            samples_file = os.path.join(samples_path, f'{prefix}.txt')
            samples = pd.DataFrame(selected_samples).transpose()
            samples.index.name = 'task'
            samples.to_csv(samples_file, sep='\t', header=True)
            
            # 创建config文件
            cols = ['Dname', 'cts', 'samples', 'MyMethod', 'OUTRIDER', 'ABEILLE', 'OUTSINGLE']
            task_config = pd.DataFrame(columns=cols)
            task_config.index.name = 'task'
            # 填充config数据
            task = 0
            task_config.loc[0, 'Dname'] = t
            task_config.loc[0, 'cts'] = ctsfile
            task_config.loc[0, 'samples'] = samples_file
            task_config.loc[0, 'MyMethod'] = os.path.join(output_dir, f'{task:03d}_mymethod.txt.gz')
            task_config.loc[0, 'OUTRIDER'] = os.path.join(output_dir, f'{task:03d}_outrider.txt.gz')
            task_config.loc[0, 'OUTSINGLE'] = os.path.join(output_dir, f'{task:03d}_outsingle.txt.gz')
            task_config.loc[0, 'ABEILLE'] = os.path.join(output_dir, f'{task:03d}_abeille.txt.gz')
            # {prefix}/{task:03d}
            
            # 保存config文件
            config_file = os.path.join(task_config_path, f'{prefix}.config')
            task_config.to_csv(config_file, sep='\t', index=False)
            
            # 存储结果
            size_configs[f'seed_{seed}'] = {
                'config': task_config,
                'selected_samples': selected_samples,
                'output_dir': output_dir
            }
        
        # 将当前样本量的所有结果存入总字典
        sampled_configs[f'size_{size}'] = size_configs

    # sampled_configs现在包含了所有抽样结果和对应的配置文件
    # 结构为：{sample_size: {seed: {'config': df, 'selected_samples': list, 'output_dir': str}}, ...}
    output_file = os.path.join(task_config_path, f'{base_prefix}_sampled_configs_dict.pkl')
    # 保存sampled_configs字典
    with open(output_file, 'wb') as f:
        pickle.dump(sampled_configs, f)
        print(f"sampled_configs已保存到: {output_file}")
    

t00_FB_s119_g11048_pct0.08
8 92
4 46
sampled_configs已保存到: /mnt/disk7t/xwj/axolotl_rev/result/dataset_nc2017kremer/task_config/t00_FB_s119_g11048_pct0.08_sampled_configs_dict.pkl
t00_FB_s119_g11048_pct0.16
16 84
8 42
sampled_configs已保存到: /mnt/disk7t/xwj/axolotl_rev/result/dataset_nc2017kremer/task_config/t00_FB_s119_g11048_pct0.16_sampled_configs_dict.pkl
t00_FB_s119_g11048_pct0.24
24 76
12 38
sampled_configs已保存到: /mnt/disk7t/xwj/axolotl_rev/result/dataset_nc2017kremer/task_config/t00_FB_s119_g11048_pct0.24_sampled_configs_dict.pkl
